In [1]:
import datajoint as dj
import numpy as np
import time

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')
ta3 = dj.create_virtual_module('ta3', 'microns_ta3')

Connecting celiib@10.28.0.34:3306


In [ ]:
ta3p100.SynapseSpineLabel()
#ta3p100.CoarseLabel()

In [ ]:
ta3.LabelKey()

In [ ]:
axon_segments = (ta3p100.Neurite() & 'neurite_type="axon"').proj(presyn='segment_id')

relevant_axon_trace_synapses = ta3p100.Synapse \
                                * ta3p100.SynapseSpineLabel() \
                                *  ta3.LabelKey().proj('description', postsynaptic_label='numeric') \
                                & axon_segments \
                                & ta3p100.CoarseLabel.proj(postsyn='segment_id')


fingerprints = dj.U('presyn').aggr(relevant_axon_trace_synapses, 
                    spine_neck='sum(description = "Spine Neck")',
                    spine_head='sum(description = "Spine Head"),
                    spine='sum(description = "Spine")',
                    total='count(*)') & 'total>4'

fingerprints_proportion = dj.U('presyn').aggr(fingerprints, 
                    spine_neck_prop='spine_neck/total',
                    spine_head_prop='spine_head/total,
                    spine_prop='spine/total')

df = pd.DataFrame(fingerprints.fetch()).set_index('presyn').astype('float')

df_unnormed = df[['spine_neck','spine_head','spine']]


df_normed = df[['spine_neck','spine_head','spine']].div(
                df[['spine_neck','spine_head','spine']].sum(axis=1), axis=0)

In [ ]:
ta3p100.ProofreadLabel() & "segment_id=648518346349472601"

In [ ]:
ta3p100.SynapseCompartmentLabel()

In [ ]:
ta3p100.SynapseCompartmentLabel() + ta3p100.SynapseSpineLabel()

In [ ]:
axon_segments = (ta3p100.Neurite() & 'neurite_type="axon"').proj(presyn='segment_id')

relevant_axon_trace_synapses = ta3p100.Synapse \
                                * ta3p100.SynapseCompartmentLabel() \
                                *  ta3.LabelKey().proj('description', postsynaptic_label='numeric') \
                                & axon_segments \
                                & ta3p100.CoarseLabel.proj(postsyn='segment_id')


fingerprints = dj.U('presyn').aggr(relevant_axon_trace_synapses, 
                    apical='sum(description = "Apical")',
                    basal='sum(description = "Basal")',
                    oblique='sum(description = "Oblique")',
                    soma='sum(description = "Soma")',
                    axon='sum(description like "Axon%%")',
                    total='count(*)') & 'total>10'



fingerprints_prop = dj.U('presyn').aggr(fingerprints, 
                    apical_prop='apical/total',
                    basal='basal/total',
                    oblique='oblique/total',
                    soma='soma/total',
                    axon='axon/total') 

df = pd.DataFrame(fingerprints.fetch()).set_index('presyn').astype('float')

df_unnormed = df[['apical','basal','oblique','soma','axon']]


df_normed = df[['apical','basal','oblique','soma','axon']].div(
                df[['apical','basal','oblique','soma','axon']].sum(axis=1), axis=0)

In [ ]:
ta3p100.SynapseSpineLabel()

In [ ]:
#create the new SPINE cgal table
@schema
class ComponentAutoSegment(dj.Computed):
    definition = """
    # creates the labels for the mesh table
    -> ta3p100.Compartment.Component
    clusters     : tinyint unsigned  #what the clustering parameter was set to
    smoothness   : decimal(3,2)             #what the smoothness parameter was set to, number betwee 0 and 1
    ---
    n_triangles  : int unsigned # number of faces
    seg_group    : longblob     # group segmentation ID's for faces from automatic CGAL segmentation
    sdf          : longblob     #  width values for faces from from automatic CGAL segmentation
    median_sdf   : decimal(6,5) # the median width value for the sdf values
    mean_sdf     : decimal(6,5) #the mean width value for the sdf values
    third_q      : decimal(6,5) #the upper quartile for the mean width values
    ninety_perc  : decimal(6,5) #the 90th percentile for the mean width values
    time_updated : timestamp    # the time at which the segmentation was performed
   
    
   """

In [ ]:
ta3.LabelKey()

In [ ]:
ta3p100.Compartment.Component() 

In [ ]:
ta3p100.Compartment.Component() & "n_triangle_indices>5000" & "compartment_type='Axon-Soma'"

In [ ]:
ta3p100.CoarseLabel() & "segment_id=648518346349471565"

In [ ]:
@schema
class ComponentLabel(dj.Computed):
    definition = """
    # creates the labels for the mesh table
    -> ta3p100.ComponentAutoSegment
    time_updated      :timestamp    # the time at which the component labels were updated
    ---
    n_vertices        :int unsigned #number of vertices in component
    n_triangles       :int unsigned #number of faces in component
    labeled_vertices  :longblob     #indicate which vertices are spine,spine_head,spine_neck otherwise 0
    labeled_triangles :longblob     #indicate which faces are spine,spine_head,spine_neck otherwise 0
    n_heads           :int unsigned #totals the number of heads after classification, helps for optimization
    used_version      :tinyint      #whether this component is used in the final labels or not, 0 no, 1 yes
    
   """

In [ ]:
ta3p100.ComponentLabel()

In [ ]:
#find that all of the neurons were processed
comp_ids = cmp.fetch("segment_id")
#print(len(np.unique(comp_ids)))
comp_unique_ids = np.unique(comp_ids)

In [ ]:
#seem to be missing 3 neurons, find out which ones those are
proof_id = (ta3p100.ProofreadLabel() & "status='complete'" & ta3p100.CoarseLabel.proj()).fetch("segment_id")
missing_ids = set(proof_id).difference(set(comp_unique_ids))

In [ ]:
missing_ids

In [ ]:
missing_ids

In [ ]:
#FIND OUT WHO THE AUTHROS WERE FOR THE MISSING IDS
key = dict(segment_id=648518346349506952,status="complete")
(ta3p100.ProofreadLabel() & key)

In [ ]:
class ComponentAutoSegment(dj.Computed):
    definition = """
    # creates the labels for the mesh table
    -> ta3p100.Compartment.Component
    clusters     : tinyint unsigned  #what the clustering parameter was set to
    smoothness   : decimal(3,2)             #what the smoothness parameter was set to, number betwee 0 and 1
    ---
    n_triangles  : int unsigned # number of faces
    seg_group    : longblob     # group segmentation ID's for faces from automatic CGAL segmentation
    sdf          : longblob     #  width values for faces from from automatic CGAL segmentation
    median_sdf   : decimal(6,5) # the median width value for the sdf values
    mean_sdf     : decimal(6,5) #the mean width value for the sdf values
    third_q      : decimal(6,5) #the upper quartile for the mean width values
    ninety_perc  : decimal(6,5) #the 90th percentile for the mean width values
    time_updated : timestamp    # the time at which the segmentation was performed
   """

In [ ]:
cas = ta3p100.ComponentAutoSegment()
seg_id = cas.fetch("segment_id")
print(len(seg_id))

In [ ]:
key_source = ta3p100.Compartment.Component & 'n_triangle_indices>100' & [dict(compartment_type=comp) for comp in ['Basal', 'Apical', 'Oblique', 'Dendrite']]
key_source

In [ ]:
ta3p100.Compartment.Component()

In [ ]:
(schema.jobs & "table_name='__component_auto_segment'").delete()